In [ ]:
import streamlit as st
import pandas as pd
import os
import pickle
from PIL import Image

# Constants
FEMALE_GENDER = 1
MALE_GENDER = 2

def load_rf_model():
    try:
        with open('random_forest_model.pkl', 'rb') as file:
            model = pickle.load(file)
        return model
    except Exception as e:
        st.error(f"Failed to load the RandomForestClassifier model: {e}")
        return None

# Load the trained model
rf_model = load_rf_model()

# Load the dataset
try:
    dataset = pd.read_csv('dataset.csv')
except Exception as e:
    st.error(f"Failed to load dataset: {e}")

def classify(gender, age, measurements):
    try:
        data = pd.DataFrame(columns=['Gender', 'Age', 'Shoulder', 'Waist', 'Hips', 'Bust', 'Chest'])
        data.loc[0] = [FEMALE_GENDER if gender == "Female" else MALE_GENDER, age] + measurements
        
        if rf_model:
            body_type = rf_model.predict(data)[0]
            st.success(f"Predicted Body Type: {body_type}")
            provide_recommendations(body_type, gender)
        else:
            st.error("RandomForestClassifier Model not loaded.")
    except Exception as e:
        st.error(str(e))

def provide_recommendations(body_type, gender):
    try:
        filtered_data = dataset[(dataset['Gender'] == gender) & (dataset[body_type] == 1)]
        
        recommendations = {}
        for cloth_pattern in filtered_data['Cloth Pattern '].unique():
            pattern_data = filtered_data[filtered_data['Cloth Pattern '] == cloth_pattern]
            top_images = pattern_data[['Image Path']].head(5).values.tolist()
            recommendations[cloth_pattern] = top_images
        
        display_recommendations(recommendations)
    except Exception as e:
        st.error(str(e))

def display_recommendations(recommendations):
    for cloth_pattern, image_paths in recommendations.items():
        st.subheader(f"Top 5 images for {cloth_pattern}:")
        cols = st.columns(len(image_paths))
        for col, image_path in zip(cols, image_paths):
            full_image_path = os.path.join(os.getcwd(), image_path[0])
            img = Image.open(full_image_path)
            col.image(img, use_column_width=True)

st.title("Body Measurement Classifier")

# Gender selection
gender = st.selectbox("Gender:", ["Female", "Male"])

# Age input
age = st.number_input("Age:", min_value=0)

# Measurement inputs
measurement_labels = ["Shoulder", "Waist", "Hips", "Bust", "Chest"]
measurements = []
for label in measurement_labels:
    measurement = st.number_input(f"{label}:", min_value=0.0)
    measurements.append(measurement)

if st.button("Classify"):
    classify(gender, age, measurements)
